# Assignment 3: Heirarchical RL: SMDP & Intra Option algorithms

In [ ]:
import numpy as np
import random
import gym
from gym.wrappers import Monitor 
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output 
from tqdm import tqdm
import time

In [ ]:
env = gym.make("Taxi-v3") # env.seed(0)
state_shape = 1 #Discrete(500) no_of_actions = env.action_space.n
print("State shape",state_shape)
print("Number of Actions", no_of_actions)
print(env.action_space.sample())
print("----")
    # Actions
    # There are 6 discrete deterministic actions:
    # - 0: move south
    # - 1: move north
    # - 2: move east
    # - 3: move west
    # - 4: pickup passenger
    # - 5: drop off passenger
state = env.reset()
''' This returns the initial state (when environment is reset) '''
print(state)
print("----")
action = env.action_space.sample() 
''' We take a random action now '''
print(action)
print("----")
next_state, reward, done, info = env.step(action)
''' env.step is used to calculate new state and obtain reward based on old state and ac tion taken '''
print(next_state)
print(reward)
print(done)
print(info)
print("----")

In [ ]:
next_state, reward, done, info = env.step(0)
env.render()
list(env.decode(next_state))

In [ ]:
route_red = [[1,3,0,0,0],
             [1,3,0,0,0],
             [1,3,3,3,3],
             [1,1,1,1,1],
             [1,1,1,1,1]]
route_green = [[0,0,2,2,1],
             [0,0,2,2,1],
             [2,2,2,2,1],
             [1,1,1,1,1],
             [1,1,1,1,1]]
route_yellow = [[0,0,0,0,0],
             [0,0,0,0,0],
             [0,3,3,3,3],
             [0,1,1,1,1],
             [0,1,1,1,1]]
route_blue = [[0,0,0,0,0],
             [0,0,0,0,0],
             [2,2,2,0,3],
             [1,1,1,0,3],
             [1,1,1,0,3]]
loc = {0: 'Red',
       1: 'Green',
       2: 'Yellow',
       3: 'Blue',
       4: 'Taxi'}

action_name = {0: 'S',
                1: 'N',
                2: 'E',
                3: 'W',
                4: 'P',
                5: 'D',
                6: 'R',
                7: 'G',
                8: 'Y',
                9: 'B'}

In [ ]:
route_map = {6: route_red,
             7: route_green,
             8: route_yellow,
             9: route_blue}
def Option(env, state, loc):
    taxi_pos = list(env.decode(next_state))
    optact = route_map[loc][taxi_pos[0]][taxi_pos[1]] optdone = False
    if loc==6 and taxi_pos[0]==0 and taxi_pos[1]==0: optdone = True if loc==7 and taxi_pos[0]==0 and taxi_pos[1]==4: optdone = True if loc==8 and taxi_pos[0]==4 and taxi_pos[1]==0: optdone = True if loc==9 and taxi_pos[0]==4 and taxi_pos[1]==3: optdone = True
return [optact, optdone]

In [ ]:
#Q-Table: (States x Actions) === (env.ns(500) x total actions(10))
q_values = np.zeros((500,10))
# TODO: epsilon-greedy action selection function
def egreedy_policy(q_values,state,epsilon):
    rg = np.random.RandomState()
    if not q_values[state].any() or rg.rand() < epsilon:
        return rg.choice(q_values.shape[-1])
    else:
        return np.argmax(q_values[state])

In [ ]:
#### SMDP Q-Learning
q_values = np.zeros((500,10))
updates = np.zeros((500,10))
rewards = []
# Add parameters you might need here
gamma = 0.9
alpha = 0.4
# Iterate over 1000 episodes
for epi in tqdm(range(2000)): 
    state = env.reset()
    done = False
    tot_reward = 0
    # While episode is not over
    while not done:
        # Choose action
        action = egreedy_policy(q_values, state, epsilon=0.1) # print(state, action)
        # Checking if primitive action
        if action < 6:
        # Perform regular Q-Learning update for state-action pair
            next_state, reward, done,_ = env.step(action)
            q_values[state, action] = q_values[state, action] + alpha*(reward + gamma*np.max(q_values[next_state, :]) - q_values[state, action])
            updates[state, action] += 1
            state = next_state
            tot_reward += reward
        # Checking if action chosen is an option
        reward_bar = 0 
        if action >= 6:
            optdone = False
            cnt = 0
            prev = state
            while (optdone == False):
                # Think about what this function might do?
                optact,optdone = Option(env,state,action)
                next_state, reward, done,_ = env.step(optact)
                # Is this formulation right? What is this term?
                reward_bar = reward_bar + reward*(gamma**cnt)
                cnt += 1
                tot_reward += reward
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                state = next_state 
                if(done): break
            q_values[prev, action] += alpha*(reward_bar + (gamma**cnt)*np.max(q_values[state, :]) - q_values[prev, action])
            updates[prev, action] += 1
    rewards.append(tot_reward)
q_values_SMDP = q_values.copy()
updates_SMDP = updates.copy()
rewards_SMDP = rewards.copy()

In [ ]:
def plot_stats(rewards, name='SMDP'): 
    plt.title(f'Reward Curve for {name}')
    plt.xlabel('Episodes') plt.ylabel('Reward')
    plt.plot(rewards) plt.savefig(f'{name}.png')
    plt.show()
    print("Average Reward for last 500 episodes", np.mean(rewards[-500:]))


plot_stats(rewards_SMDP, 'SMDP')

In [ ]:
def Qmatrix(q_values, i, j): 
    qmatrix = np.zeros((5,5,10)) 
    for row in range(5):
        for col in range(5):
            qmatrix[row][col][:] = np.copy(q_values[env.encode(row,col,i,j)])
    return qmatrix

def vizQ(q_values): 
    for i in range(5):
        for j in range(4):
            if i == j: continue
            print(f"Passenger at {loc[i]} going to {loc[j]}") 
            for row in range(5):
                for col in range(5): 
                    print(action_name[np.argmax(q_values[env.encode(row,col,i,j)])],end=" ")
                print()

In [ ]:
UP = 1
DOWN = 0
LEFT = 3
RIGHT = 2
def plot_Q(q_values):
    fig, axs = plt.subplots(5, 4, figsize=(50,50))
    for i in range(5):
        for j in range(4):
            #if i==j: continue
            Q = Qmatrix(q_values_SMDP,i,j)
            #fig, ax = plt.subplots()
            # plt.figure(figsize=(5,5))
            axs[i, j].set_title(f"Passenger at {loc[i]} going to {loc[j]}") 
            axs[i, j].invert_yaxis()

            axs[i, j].pcolor(Q.max(-1), edgecolors='k', linewidths=2) 
            # plt.colorbar()
            def x_direct(a):
            if a == RIGHT: return 1 if a == LEFT: return -1 return 0
            
            def y_direct(a):
            if a == UP: return 1 if a == DOWN: return -1 return 0
            policy = Q.argmax(-1)
            policyx = np.vectorize(x_direct)(policy)
            policyy = np.vectorize(y_direct)(policy)
            idx = np.indices(policy.shape)
            axs[i, j].quiver(idx[1].ravel()+0.5, idx[0].ravel()+0.5, policyx.ravel(), policyy.ravel(), pivot="middle", color='red')
        
            for k in range(25):
                act = policy.reshape(25,)[k]
                if act < 4: continue
                axs[i, j].annotate(f'{action_name[act]}', xy=(idx[1].ravel()[k]+0.5,idx[0].ravel()[k]+0.5), fontsize=20, ha='center', va='center')
        
    plt.show()
pltQ = plot_Q(q_values_SMDP)

In [ ]:
#### Intra-option Q-Learning
q_values = np.zeros((500,10))
updates = np.zeros((500,10))
rewards = []
# Add parameters you might need here
gamma = 0.9
alpha = 0.5
# Iterate over 1000 episodes
for _ in tqdm(range(2000)):
    state = env.reset() 
    done = False 
    tot_reward = 0
    # While episode is not over
    while not done:
        # Choose action
        action = egreedy_policy(q_values, state, epsilon=0.1) # print(state, action)
        # Checking if primitive action
        if action < 6:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done,_ = env.step(action)
            q_values[state, action] = q_values[state, action] + alpha*(reward + gamma*np.max(q_values[next_state, :]) - q_values[state, action])
            updates[state, action] += 1
            state = next_state
            tot_reward += reward
            # Checking if action chosen is an option 
        if action >= 6: # action => Top option
            optdone = False
            cnt = 0
            prev = state
            while (optdone == False):
                # Think about what this function might do?
                optact,optdone = Option(env,state,action)
                next_state, reward, done,_ = env.step(optact)
                tot_reward += reward
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                optact_tmp,optdone_tmp = Option(env,next_state,action)
                q_values[state, optact] += alpha*(reward + gamma*np.max(q_values[next_state, :]) - q_values[state, optact])
                updates[state, optact] += 1 
                if(optdone_tmp):
                    q_values[state, action] += alpha*(reward + gamma*np.max(q_values[next_state, :]) - q_values[state, action])
                    updates[state, action] += 1
                    state = next_state
                    break 
                else:
                    q_values[state, action] += alpha*(reward + gamma*q_values[next_state, action] - q_values[state, action])
                    updates[state, action] += 1 
                state = next_state
                if(done): 
                    break
    rewards.append(tot_reward)
q_values_intra_option = q_values.copy()
updates_intra_option = updates.copy()
rewards_intra_option = rewards.copy()

In [ ]:
plot_stats(rewards_intra_option, 'Intra Option')
pltQ = plot_Q(q_values_intra_option)